In [5]:
import pandas as pd

In [6]:
df = pd.read_csv("new_data.csv")
df.columns

Index(['event id', 'event name', 'EVENT_START_DATETIME',
       'CLASSIFICATION_GENRE', 'CLASSIFICATION_SUB_GENRE', 'MIN_PRICE',
       'MAX_PRICE', 'ATTRACTION_NAME', 'attraction id', 'venue city',
       'venue id', 'venue address', 'venue state', 'VENUE_TIMEZONE',
       'classification type', 'classification subtype',
       'classification segment', 'classification genre',
       'classification subgenre', 'classification type.1',
       'classification subtype.1', 'scrape time'],
      dtype='object')

In [7]:
min_price_df = df[["event id", "MIN_PRICE", "scrape time"]]
earliest_times = min_price_df.drop("MIN_PRICE", axis=1).groupby(by="event id").min().reset_index()
earliest_prices = min_price_df.merge(earliest_times, on=["event id", "scrape time"], how="right")

latest_times = min_price_df.drop("MIN_PRICE", axis=1).groupby(by="event id").max().reset_index()
latest_prices = min_price_df.merge(latest_times, on=["event id", "scrape time"], how="right")

diff_min = earliest_prices.merge(latest_prices, on="event id", how="inner", suffixes=["_early", "_late"])
diff_min["markup"] = diff_min["MIN_PRICE_late"] - diff_min["MIN_PRICE_early"]
diff_min.head()

,event id,MIN_PRICE_early,scrape time_early,MIN_PRICE_late,scrape time_late,markup
0,11AYvxaAAFjtv5,59.98,2025-02-07 20:36:49.127253,59.98,2025-02-07 20:36:49.127253,0.0
1,11FZv-oF-4ANv5,29.50,2025-02-07 20:36:47.970969,29.50,2025-02-08 18:16:58.843231,0.0
2,169ZkAx8KZA51Ckv,78.00,2025-02-07 20:36:47.955347,78.00,2025-02-07 20:36:47.955347,0.0
3,169ZkAx8KZAG1CkA,78.00,2025-02-07 20:36:47.955347,78.00,2025-02-14 05:00:02.126734,0.0
4,169ZkAx8KZAG1Ckd,79.00,2025-02-07 20:36:47.955347,79.00,2025-02-12 02:00:02.851363,0.0


In [8]:
diff_min[diff_min["markup"] != 0].shape[0] / diff_min.shape[0]

0.06313993174061433

The above shows only 6% of all concerts had any markup/discount over time at all.

In [9]:
num_prices = df.groupby("event id").nunique()
num_prices[num_prices["MIN_PRICE"] > 2][["MIN_PRICE"]].shape[0]

2

The above shows that only 2 concerts in our entire dataset of several hundred concerts had more than 2 unique minimum ticket prices over the time interval that we scraped their data. Of the few concerts that changed price, only 2 changed price more than once.

In [10]:
just_event_and_price = df.dropna(subset="MAX_PRICE")[["event id", "MAX_PRICE"]]
stds = just_event_and_price.groupby("event id").std()
had_change = stds[stds["MAX_PRICE"] > 0]
had_change.shape[0] / stds.shape[0]

0.06324786324786325

The above uses the standard deviation of all maximum ticket prices for each concert to show that only 6% of the concerts in our dataset have any change in maximum ticket price.